# ios上架有一堆解析度需要調整，這裡做個工具幫忙調整

## MAC FFMPEG 安裝方式
```
brew install ffmpeg
```
https://juejin.cn/post/6844904191433900040
https://stackoverflow.com/questions/54161418/python3-6-add-audio-to-cv2-processed-video

6.5 吋 886 x 1920 像素（直向）  
5.5 吋  1080 x 1920   
4.7 吋   750 x 1334 像素  
12.9 吋   1200 x 1600   
9.7 吋    900 x 1200   

## 函式庫先執行

In [15]:
#opencv內建的Resize 比例會跑掉，這是用裁切的方式等比例放大或是縮小
def AkingResize(img , OutPutWidth , OutPutHeight):
    InimgW = img.shape[1]
    InimgH = img.shape[0]
    InRatio = InimgW / InimgH
    OutRatio = OutPutWidth / OutPutHeight
    if (OutRatio > InRatio): #更扁平 所以縮放的時候寬度一樣，原圖的高度要裁切
        #print ("壓扁")
        NewHeight = int(InimgW * OutPutHeight / OutPutWidth)
        CenterY = InimgH / 2
        StartY = int(CenterY - NewHeight / 2)
        EndY = StartY + NewHeight
        NewImg = img[StartY:EndY,0:InimgW]
        Resizedimage = cv2.resize(NewImg, (OutPutWidth, OutPutHeight), interpolation=cv2.INTER_CUBIC)
    elif (OutRatio < InRatio): #更高長 所以縮放的時候高度一樣，原圖的寬度要裁切
        #print ("長高")
        NewWidth = int(InimgH * OutPutWidth / OutPutHeight)
        CenterX = InimgW / 2
        StartX = int(CenterX - NewWidth / 2)
        EndX = StartX + NewWidth
        NewImg = img[0:InimgH,StartX:EndX]
        Resizedimage = cv2.resize(NewImg, (OutPutWidth, OutPutHeight), interpolation=cv2.INTER_CUBIC)
    else: #相同比例
        Resizedimage = cv2.resize(img, (OutPutWidth, OutPutHeight), interpolation=cv2.INTER_CUBIC)
    return Resizedimage

## 照片調整

In [16]:
import cv2
#InputPicPathName = r'/Users/aking/Desktop/Temp/886/6.png'  #輸入影像的路徑與檔名
#OutPutWidth = 1242             #輸出寬度
#OutPutHeight = 2688                #輸出高度
#OutputPicPathName = r'/Users/aking/Desktop/Temp/1242/6.png'   #輸出影像的路徑與檔名

#1242 2688. 
#1242 2208. 
#2048 2732
#InputPicPathName = r'/Users/aking/Desktop/Temp/886/7.png'  #輸入影像的路徑與檔名
#OutPutWidth = 2048             #輸出寬度
#OutPutHeight = 2732                #輸出高度
#OutputPicPathName = r'/Users/aking/Desktop/Temp/2732/7.png'   #輸出影像的路徑與檔名

def ResizePic(InputPicPathName , OutPutWidth,OutPutHeight,OutputPicPathName):
    #=============開始轉換====================================
    img = cv2.imread(InputPicPathName)
    if (img.any() != None):
        Resizedimage = AkingResize(img , OutPutWidth , OutPutHeight)
        cv2.imwrite(OutputPicPathName, Resizedimage)
        # 顯示圖片
        #cv2.imshow('Resized image', Resizedimage)
        #cv2.waitKey(3)
        #cv2.destroyAllWindows()
        print ("圖片轉換完成")
        # 按下任意鍵則關閉所有視窗
    else:
        print ("圖片讀取失敗")

In [22]:
from os import listdir
from os.path import isfile, isdir, join
import os
sourcePath = r'/Users/aking/Desktop/Temp/Pic'

def DoPic(sourcePath ,OutPutWidth ,OutPutHeight):
    files = listdir(sourcePath)
    OutputPicPath = join(sourcePath ,"{}".format(OutPutHeight ))
    if not os.path.isdir(OutputPicPath):
        os.mkdir(OutputPicPath)
    for file in files:
        #先搞定資料夾
        print (file)
        if (file == ".DS_Store"):
            continue
        InputPicPathName = join(sourcePath ,file )
        if (os.path.isfile(InputPicPathName)):
            OutputPicPathName = join(OutputPicPath ,file )
            ResizePic(InputPicPathName , OutPutWidth,OutPutHeight,OutputPicPathName)
        
DoPic(sourcePath , 1242 ,2688 )   
DoPic(sourcePath , 1242 ,2208 ) 
DoPic(sourcePath , 2048 ,2732 ) 

#1242 2688. 
#1242 2208. 
#2048 2732

圖片轉換完成
圖片轉換完成
圖片轉換完成
圖片轉換完成
圖片轉換完成
圖片轉換完成
圖片轉換完成
圖片轉換完成
圖片轉換完成
圖片轉換完成
圖片轉換完成
圖片轉換完成
圖片轉換完成
圖片轉換完成
圖片轉換完成
圖片轉換完成
圖片轉換完成
圖片轉換完成
圖片轉換完成
圖片轉換完成
圖片轉換完成
圖片轉換完成
圖片轉換完成
圖片轉換完成


## 影片調整

In [4]:
import cv2
import os

def ResizeVideo(InputVideoPathName , OutPutWidth,OutPutHeight,OutputVideoPathName):
    #=======================輸入設定====================================
    video = cv2.VideoCapture(InputVideoPathName)
    # 取得影像的尺寸大小
    width = video.get(cv2.CAP_PROP_FRAME_WIDTH)
    height = video.get(cv2.CAP_PROP_FRAME_HEIGHT)
    fps = video.get(cv2.CAP_PROP_FPS) #每秒鐘幾張
    print ("原始寬度{} 高度{} FPS{}".format(width , height,fps))


    #=======================輸出檔案設定=======================
    fourcc = cv2.VideoWriter_fourcc(*'avc1')
    VideoFilePath = r"{}/TmpVideo.mp4".format(os.path.dirname(InputVideoPathName)) #解析度調整後的畫面
    SoundFilePath = r"{}/sound.mp3".format(os.path.dirname(InputVideoPathName))    #原始檔案分離出聲音
    VideoWithSoundFilePath = r"{}/TmpVideoWithSound.mp4".format(os.path.dirname(InputVideoPathName)) #畫面與聲音合成影片
    #畫面與聲音合成影片 之後需要再轉檔一次 app store才會接受

    out = cv2.VideoWriter(VideoFilePath, fourcc, fps, (int(OutPutWidth),  int(OutPutHeight)))
    #=============開始轉換====================================
    while video.isOpened():
        ret, frame = video.read()
        if not ret:
            break
        image = AkingResize(frame, OutPutWidth, OutPutHeight)
        out.write(image)
        cv2.imshow('frame', image)
        if cv2.waitKey(1) == ord('q'):
            break
    video.release()
    out.release()

    CMD = r"ffmpeg -i {} -f mp3 -ab 256000 -vn {}".format(InputVideoPathName,SoundFilePath) #原始檔案分離出聲音
    os.system(CMD) #原始檔案分離出聲音


    CMD1 = r"ffmpeg -i {} -i {} -c copy -map 0:v:0 -map 1:a:0 {}".format(VideoFilePath, SoundFilePath , VideoWithSoundFilePath)
    os.system(CMD1) #畫面與聲音合成影片

    CMD2 = "ffmpeg -i {} -filter_complex 'scale={}:{},framerate=30' {}".format(VideoWithSoundFilePath , OutPutWidth,OutPutHeight,OutputVideoPathName)
    os.system(CMD2) #最後再轉檔案一次 App store才會吃



    os.system("rm -f {}".format(VideoFilePath)) 
    os.system("rm -f {}".format(SoundFilePath)) 
    os.system("rm -f {}".format(VideoWithSoundFilePath)) 



    print("轉換完成")
    cv2.destroyAllWindows()
    

In [5]:
InputVideoPathName = r'/Users/aking/Desktop/Temp/付款示範.mov' #輸入影像的路徑與檔名
OutPutWidth = 886           #輸出寬度
OutPutHeight = 1920                #輸出高度
OutputVideoPathName = r'/Users/aking/Desktop/Temp/付款示範{}.mp4'.format(OutPutWidth)   #輸出影像的路徑與檔名
ResizeVideo(InputVideoPathName,OutPutWidth,OutPutHeight,OutputVideoPathName)

原始寬度1170.0 高度2532.0 FPS45.54374543725344


ffmpeg version 5.0 Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 13.0.0 (clang-1300.0.29.30)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/5.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --disable-indev=jack --enable-videotoolbox
  lib

轉換完成


frame=  672 fps= 65 q=-1.0 Lsize=    4612kB time=00:00:22.30 bitrate=1694.4kbits/s speed=2.14x    
video:4582kB audio:6kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.537162%
[libx264 @ 0x7facd7708d80] frame I:7     Avg QP:15.46  size: 32551
[libx264 @ 0x7facd7708d80] frame P:197   Avg QP:19.43  size: 12812
[libx264 @ 0x7facd7708d80] frame B:468   Avg QP:20.54  size:  4144
[libx264 @ 0x7facd7708d80] consecutive B-frames:  4.8%  5.7%  4.5% 85.1%
[libx264 @ 0x7facd7708d80] mb I  I16..4: 47.1% 44.8%  8.1%
[libx264 @ 0x7facd7708d80] mb P  I16..4:  8.7% 11.0%  1.8%  P16..4: 13.5%  2.5%  1.2%  0.0%  0.0%    skip:61.4%
[libx264 @ 0x7facd7708d80] mb B  I16..4:  1.1%  1.5%  0.1%  B16..8: 13.1%  1.4%  0.2%  direct: 1.8%  skip:80.8%  L0:45.8% L1:51.2% BI: 2.9%
[libx264 @ 0x7facd7708d80] 8x8 transform intra:51.4% inter:86.2%
[libx264 @ 0x7facd7708d80] coded y,uvDC,uvAC intra: 27.0% 30.0% 6.0% inter: 4.6% 5.7% 0.1%
[libx264 @ 0x7facd7708d80] i16 v,h,dc,p: 60% 26%  5% 10%
[li

In [6]:
InputVideoPathName = r'/Users/aking/Desktop/Temp/GiantShow.mp4' #輸入影像的路徑與檔名
OutPutWidth = 886           #輸出寬度
OutPutHeight = 1920                #輸出高度
OutputVideoPathName = r'/Users/aking/Desktop/Temp/GiantShow{}.mp4'.format(OutPutWidth)   #輸出影像的路徑與檔名
ResizeVideo(InputVideoPathName,OutPutWidth,OutPutHeight,OutputVideoPathName)

OutPutWidth = 1080           #輸出寬度
OutPutHeight = 1920                #輸出高度
OutputVideoPathName = r'/Users/aking/Desktop/Temp/GiantShow{}.mp4'.format(OutPutWidth)   #輸出影像的路徑與檔名
ResizeVideo(InputVideoPathName,OutPutWidth,OutPutHeight,OutputVideoPathName)

OutPutWidth = 1200          #輸出寬度
OutPutHeight = 1600                #輸出高度
OutputVideoPathName = r'/Users/aking/Desktop/Temp/GiantShow{}.mp4'.format(OutPutWidth)   #輸出影像的路徑與檔名
ResizeVideo(InputVideoPathName,OutPutWidth,OutPutHeight,OutputVideoPathName)

原始寬度498.0 高度1080.0 FPS30.0


ffmpeg version 5.0 Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 13.0.0 (clang-1300.0.29.30)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/5.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --disable-indev=jack --enable-videotoolbox
  lib

frame=  891 fps= 37 q=-1.0 Lsize=   15869kB time=00:00:29.73 bitrate=4371.5kbits/s speed=1.23x    
video:15828kB audio:8kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.206150%
[libx264 @ 0x7fc49c705440] frame I:14    Avg QP:19.86  size: 75976
[libx264 @ 0x7fc49c705440] frame P:337   Avg QP:23.91  size: 28888
[libx264 @ 0x7fc49c705440] frame B:540   Avg QP:25.47  size: 10016
[libx264 @ 0x7fc49c705440] consecutive B-frames: 13.7% 13.5%  9.1% 63.7%
[libx264 @ 0x7fc49c705440] mb I  I16..4: 17.2% 69.2% 13.6%
[libx264 @ 0x7fc49c705440] mb P  I16..4:  9.0% 22.9%  2.1%  P16..4: 30.1% 10.5%  4.0%  0.0%  0.0%    skip:21.4%
[libx264 @ 0x7fc49c705440] mb B  I16..4:  1.7%  2.6%  0.2%  B16..8: 33.7%  4.6%  0.7%  direct: 3.4%  skip:53.2%  L0:43.6% L1:51.0% BI: 5.5%
[libx264 @ 0x7fc49c705440] 8x8 transform intra:66.0% inter:80.4%
[libx264 @ 0x7fc49c705440] coded y,uvDC,uvAC intra: 32.7% 37.2% 9.2% inter: 15.2% 14.5% 0.3%
[libx264 @ 0x7fc49c705440] i16 v,h,dc,p: 40% 39% 10% 11%


轉換完成
原始寬度498.0 高度1080.0 FPS30.0


ffmpeg version 5.0 Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 13.0.0 (clang-1300.0.29.30)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/5.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --disable-indev=jack --enable-videotoolbox
  lib

...

轉換完成
原始寬度498.0 高度1080.0 FPS30.0


ffmpeg version 5.0 Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 13.0.0 (clang-1300.0.29.30)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/5.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --disable-indev=jack --enable-videotoolbox
  lib

[Parsed_framerate_1 @ 0x7fb8ed906e80] time base:1/15360 -> 1/15360 exact:1
[Parsed_framerate_1 @ 0x7fb8ed906e80] fps -> fps:30/1 scene score:8.200000 interpolate start:15 end:240
[libx264 @ 0x7fb8ec405700] using SAR=1/1
[libx264 @ 0x7fb8ec405700] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x7fb8ec405700] profile High, level 4.0, 4:2:0, 8-bit
[libx264 @ 0x7fb8ec405700] 264 - core 163 r3060 5db6aa6 - H.264/MPEG-4 AVC codec - Copyleft 2003-2021 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=-2 threads=12 lookahead_threads=2 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=1 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=25 scenecut=40 intra_refresh=0 rc_lookahead=40 rc=crf mbtree

轉換完成


frame=  891 fps= 29 q=-1.0 Lsize=   18237kB time=00:00:29.73 bitrate=5023.7kbits/s speed=0.97x    
video:18196kB audio:8kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.178343%
[libx264 @ 0x7fb8ec405700] frame I:14    Avg QP:20.22  size: 69950
[libx264 @ 0x7fb8ec405700] frame P:349   Avg QP:24.27  size: 32264
[libx264 @ 0x7fb8ec405700] frame B:528   Avg QP:25.27  size: 12108
[libx264 @ 0x7fb8ec405700] consecutive B-frames: 14.6% 15.3% 11.8% 58.4%
[libx264 @ 0x7fb8ec405700] mb I  I16..4: 15.5% 75.0%  9.4%
[libx264 @ 0x7fb8ec405700] mb P  I16..4:  8.8% 28.6%  1.9%  P16..4: 30.1%  9.2%  2.9%  0.0%  0.0%    skip:18.5%
[libx264 @ 0x7fb8ec405700] mb B  I16..4:  2.0%  3.3%  0.1%  B16..8: 35.0%  4.6%  0.6%  direct: 4.0%  skip:50.4%  L0:44.5% L1:50.7% BI: 4.8%
[libx264 @ 0x7fb8ec405700] 8x8 transform intra:70.9% inter:84.5%
[libx264 @ 0x7fb8ec405700] coded y,uvDC,uvAC intra: 36.7% 38.9% 8.5% inter: 16.7% 16.4% 0.3%
[libx264 @ 0x7fb8ec405700] i16 v,h,dc,p: 41% 35% 10% 14%


# 懷舊程式碼

In [26]:
ffmpeg -i /Users/aking/Documents/2021Unity/LAVA/上架檔案/影片/Lava.mov -filter_complex "scale=1200:1600,framerate=30;"  /Users/aking/Documents/2021Unity/LAVA/上架檔案/影片/ffout.mp4
ffmpeg -i /Users/aking/Documents/2021Unity/LAVA/上架檔案/影片/Lava.mov -filter_complex "[0:v]setpts=1*PTS,scale=1200:1600,framerate=30[v];[0:a]atempo=8.0[a]" -map "[v]" -map "[a]" /Users/aking/Documents/2021Unity/LAVA/上架檔案/影片/ffout1600.mp4

ffmpeg -i /Users/aking/Documents/2021Unity/LAVA/上架檔案/影片/Tmp.mp4 -i /Users/aking/Documents/2021Unity/LAVA/上架檔案/影片/sound.aac -c copy -map 0:v:0 -map 1:a:0 /Users/aking/Documents/2021Unity/LAVA/上架檔案/影片/Lava886S.mp4


In [46]:
import cv2
import os

#InputVideoPathName = r'/Users/aking/Documents/2021Unity/LAVA/上架檔案/影片/RestoreAndPurchases.mov'  #輸入影像的路徑與檔名
#OutPutWidth = 560            #輸出寬度
#OutPutHeight = 1200                 #輸出高度
#OutputVideoPathName = r'/Users/aking/Documents/2021Unity/LAVA/上架檔案/影片/RestoreAndPurchases.mp4'   #輸出影像的路徑與檔名



InputVideoPathName = r'/Users/aking/Desktop/Temp/Purchase_RestorePurchases.mov' #輸入影像的路徑與檔名
OutPutWidth = 886           #輸出寬度
OutPutHeight = 1920                #輸出高度
OutputVideoPathName = r'/Users/aking/Desktop/Temp/Purchase_RestorePurchases{}.mp4'.format(OutPutWidth)   #輸出影像的路徑與檔名


#=======================輸入設定====================================
video = cv2.VideoCapture(InputVideoPathName)
# 取得影像的尺寸大小
width = video.get(cv2.CAP_PROP_FRAME_WIDTH)
height = video.get(cv2.CAP_PROP_FRAME_HEIGHT)
fps = video.get(cv2.CAP_PROP_FPS) #每秒鐘幾張
print ("原始寬度{} 高度{} FPS{}".format(width , height,fps))


#=======================輸出檔案設定=======================
fourcc = cv2.VideoWriter_fourcc(*'avc1')
VideoFilePath = r"{}/TmpVideo.mp4".format(os.path.dirname(InputVideoPathName)) #解析度調整後的畫面
SoundFilePath = r"{}/sound.mp3".format(os.path.dirname(InputVideoPathName))    #原始檔案分離出聲音
VideoWithSoundFilePath = r"{}/TmpVideoWithSound.mp4".format(os.path.dirname(InputVideoPathName)) #畫面與聲音合成影片
#畫面與聲音合成影片 之後需要再轉檔一次 app store才會接受

out = cv2.VideoWriter(VideoFilePath, fourcc, fps, (int(OutPutWidth),  int(OutPutHeight)))
#=============開始轉換====================================
while video.isOpened():
    ret, frame = video.read()
    if not ret:
        break
    image = AkingResize(frame, OutPutWidth, OutPutHeight)
    out.write(image)
    cv2.imshow('frame', image)
    if cv2.waitKey(1) == ord('q'):
        break
video.release()
out.release()

CMD = r"ffmpeg -i {} -f mp3 -ab 256000 -vn {}".format(InputVideoPathName,SoundFilePath) #原始檔案分離出聲音
os.system(CMD) #原始檔案分離出聲音


CMD1 = r"ffmpeg -i {} -i {} -c copy -map 0:v:0 -map 1:a:0 {}".format(VideoFilePath, SoundFilePath , VideoWithSoundFilePath)
os.system(CMD1) #畫面與聲音合成影片

CMD2 = "ffmpeg -i {} -filter_complex 'scale={}:{},framerate=30' {}".format(VideoWithSoundFilePath , OutPutWidth,OutPutHeight,OutputVideoPathName)
os.system(CMD2) #最後再轉檔案一次 App store才會吃



os.system("rm -f {}".format(VideoFilePath)) 
os.system("rm -f {}".format(SoundFilePath)) 
os.system("rm -f {}".format(VideoWithSoundFilePath)) 



print("轉換完成")
cv2.destroyAllWindows()

原始寬度1170.0 高度2532.0 FPS41.09424851746791


ffmpeg version 5.0 Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 13.0.0 (clang-1300.0.29.30)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/5.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --disable-indev=jack --enable-videotoolbox
  lib

轉換完成


frame=  599 fps= 67 q=-1.0 Lsize=    3716kB time=00:00:20.13 bitrate=1511.4kbits/s speed=2.24x    
video:3687kB audio:6kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.611658%
[libx264 @ 0x7fe601205b80] frame I:7     Avg QP:17.46  size: 36329
[libx264 @ 0x7fe601205b80] frame P:181   Avg QP:18.80  size: 10009
[libx264 @ 0x7fe601205b80] frame B:411   Avg QP:18.42  size:  4159
[libx264 @ 0x7fe601205b80] consecutive B-frames:  6.3%  3.3%  9.5% 80.8%
[libx264 @ 0x7fe601205b80] mb I  I16..4: 43.5% 46.8%  9.7%
[libx264 @ 0x7fe601205b80] mb P  I16..4: 10.9% 14.3%  1.4%  P16..4: 11.5%  1.8%  0.8%  0.0%  0.0%    skip:59.3%
[libx264 @ 0x7fe601205b80] mb B  I16..4:  1.4%  1.3%  0.2%  B16..8: 13.7%  1.5%  0.2%  direct: 1.9%  skip:79.9%  L0:47.2% L1:49.9% BI: 2.9%
[libx264 @ 0x7fe601205b80] 8x8 transform intra:51.5% inter:86.3%
[libx264 @ 0x7fe601205b80] coded y,uvDC,uvAC intra: 20.2% 29.1% 5.1% inter: 3.6% 5.3% 0.1%
[libx264 @ 0x7fe601205b80] i16 v,h,dc,p: 57% 26%  6% 10%
[li